## Tensors:

In mathematics, a tensor is a geometric object that maps in a multi-linear manner geometric vectors, scalars, and other tensors to a resulting tensor. Vectors and scalars which are often used in elementary physics and engineering applications, are considered as the simplest tensors. Vectors from the dual space of the vector space, which supplies the geometric vectors, are also included as tensors.[1] Geometric in this context is chiefly meant to emphasize independence of any selection of a coordinate system.[more](https://en.wikipedia.org/wiki/Tensor)

In simple terms, **Tensor** is an arrangment of number in different dimentions. For example
<br>
- Vectors/Arrays are 1D-Tensor.
- Matrix with rows and coulmns are 2D-Tensor.
- RGB image is an example of 3D-Tensor.  

Similarly we can have 4D-Tensor, 5D-Tensor ..... N-D-Tensor etc.

Throught the rest of our discussion we will see how we can use **Pytorch's python and C++ frontend** to create these **Tensors** and how we can playaround and work with Pythorch API's to implement a simple Neural net to most complicated ones.

## Tensor creation using Pytorch 

### using Python frontend:


```python
# First, import PyTorch
import torch

### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 5 random normal variables
features = torch.randn((1, 5))
print(features)

############# OUTPUT #################

tensor([[-0.1468,  0.7861,  0.9468, -1.1143,  1.6908]])

```

### using C++ frontend:
```cpp
#include <torch/torch.h>
#include <ATen/Context.h>
#include <iostream>

int main()
{
  // Seed for Random number
  torch::manual_seed(9);
  torch::Tensor features = torch::rand({1, 5});
  std::cout << "features = " << features << "\n";
}

################ OUTPUT #######################

features =  0.6558  0.3020  0.4799  0.7774  0.9180
[ Variable[CPUFloatType]{1,5} ]

```

As we can see we include the headerfiles which is similar to importing a module in python.  
**NOTE**: We are using similar API's **manual_seed() and rand()** to generate the Tensor. This is a pattern which we will see and can be used as and advantage to convert our python code to c++ or viceversa.

## Get the Shape of Tensor

### Using Python:

Pytoch syntax is pretty similar to Numpy.

```python
print(features.shape)
print(features.shape[0])
print(features.shape[1])

############# OUTPUT #################

torch.Size([1, 5])
1
5

```

### Using C++:

```cpp
#include <torch/torch.h>
#include <ATen/Context.h>
#include <iostream>

int main()
{
  // Seed for Random number
  torch::manual_seed(9);
  torch::Tensor features = torch::rand({1, 5});
  std::cout << "features shape = " << features.sizes() << "\n";
  std::cout << "features shape rows = " << features.sizes()[0] << "\n";
  std::cout << "features shape col = " << features.size(1) << "\n";

}

################ OUTPUT #######################

features shape = [1, 5]
features shape rows = 1
features shape col = 5

```

Notice the different ways/methods of accessing the indexes of the Tensors.
```cpp
  // Numpy like indexing style.
  std::cout << "features shape rows = " << features.sizes()[0] << "\n";
  // C++ way of accessing the indexes.
  std::cout << "features shape col = " << features.size(1) << "\n";
```

In the rest of the of the discussion, we use the below version as our choice to access the shape of the **Tensor**.
```cpp
  std::cout << "features shape col = " << features.size(1) << "\n";
```




## Simple Neural Net.

Let us create a simple Neual Net from scratch using Pytorch. 

<img src="assets/simple_neuron.png" width=400px>

Mathematically this looks like: 

$$
\Large
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i +b \right)
\end{align}
$$

The above mathemetical formula can also be written in matrix form as follows.

$$
\Large
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

### Using Python:

In [24]:
# First, import PyTorch
import torch

In [25]:
def ActivationFunction(x):
    """ Sigmoid activation function 
    
        Arguments
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

In [26]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 5 random normal variables
features = torch.randn((1, 5))
# True weights for our data, random normal variables again
weights = torch.randn_like(features)
# and a true bias term
bias = torch.randn((1, 1))

In [27]:
print(features.shape)
print(features.shape[0])
print(features.shape[1])

torch.Size([1, 5])
1
5


In [28]:
y = ActivationFunction(torch.sum(features * weights) + bias)
features.shape, weights.shape, bias.shape, y

(torch.Size([1, 5]),
 torch.Size([1, 5]),
 torch.Size([1, 1]),
 tensor([[0.1595]]))

In [29]:
y = ActivationFunction((features*weights).sum() + bias)
features.shape, weights.shape, bias.shape, y

(torch.Size([1, 5]),
 torch.Size([1, 5]),
 torch.Size([1, 1]),
 tensor([[0.1595]]))

In [30]:
y = ActivationFunction(torch.mm(features,weights.view(5,1)) + bias)
y

tensor([[0.1595]])

### Using C++:

```cpp
#include <torch/torch.h>
#include <ATen/Context.h>
#include <iostream>

torch::Tensor ActivationFunction(const torch::Tensor &x)
{
  // Sigmoid function.
  auto retVal = 1/(1+torch::exp(-x));
  return retVal;
}

int main()
{
  // Seed for Random number
  torch::manual_seed(9);
  //at::manual_seed(9);

  torch::Tensor features = torch::rand({1, 5});
  std::cout << "features = " << features << "\n";

  auto weights = torch::randn_like(features);
  std::cout << "weights = " << weights << "\n";

  auto bias = torch::randn({1,1});
  std::cout << "bias  =" << bias << "\n";

  // There are multiple ways to get the same result. Here are the few of them.
  // 1st way
  auto y = ActivationFunction(torch::sum(features * weights) +bias);
  std::cout << "y using (\"torch::sum()\") = " << y << "\n";

  // 2nd way
  y = ActivationFunction((features * weights).sum() +bias);
  std::cout << "y using (\".sum()\") = " << y << "\n";

  // 3rd and preferred way. Using Matrix multiplication.
  y = ActivationFunction(torch::mm(features, weights.view({5,1}))+bias);
  std::cout << "y using (\"torch::mm().view()\") = " << y << "\n";

  // Reshaping the tensor using reshape().
  y = ActivationFunction(torch::mm(features, weights.reshape({5,1}))+bias);
  std::cout << "y using (\"torch.mm().reshape() \") = " << y << "\n";

  // Reshaping the tensor using inplace resize_().
  y = ActivationFunction(torch::mm(features, weights.resize_({5,1}))+bias);
  std::cout << "y using (\"torch.mm().resize_() \") = " << y << "\n";

  std::cout << "features shape = " << features.sizes() << "\n";
  std::cout << "features shape rows = " << features.sizes()[0] << "\n";
  std::cout << "features shape col = " << features.size(1) << "\n";
  
  return 0;
}


################ OUTPUT #######################

features =  0.6558  0.3020  0.4799  0.7774  0.9180
[ Variable[CPUFloatType]{1,5} ]
    
weights = -1.3316  0.4487 -0.2635  1.2342 -1.1583
[ Variable[CPUFloatType]{1,5} ]
    
bias  =-1.7026
[ Variable[CPUFloatType]{1,1} ]
    
y using ("torch::sum()") = 0.01 * 6.4732
[ Variable[CPUFloatType]{1,1} ]
    
y using (".sum()") = 0.01 * 6.4732
[ Variable[CPUFloatType]{1,1} ]
    
y using ("torch::mm().view()") = 0.01 * 6.4732
[ Variable[CPUFloatType]{1,1} ]
    
y using ("torch.mm().reshape() ") = 0.01 * 6.4732
[ Variable[CPUFloatType]{1,1} ]
    
y using ("torch.mm().resize_() ") = 0.01 * 6.4732
[ Variable[CPUFloatType]{1,1} ]
    
features shape = [1, 5]
features shape rows = 1
features shape col = 5


```

## Stacked Neural Net.
<img src='assets/multilayer_diagram_weights.png' width=450px>